# Neuron Network Model 

In [1]:
import numpy as np
import pickle
import pandas as pd

## Preprocessing

In [2]:
with open('fstAgg.pkl','rb') as handle:
    unserialized_data = pickle.load(handle)

In [3]:
X_train_df = unserialized_data['X_train_df']
X_test_df = unserialized_data['X_test_df']
y_train_df = unserialized_data['y_train_df']
y_test_df = unserialized_data['y_test_df']

In [4]:
X_train_df = X_train_df.astype("float")
X_test_df = X_test_df.astype("float")

In [5]:
y_train_df.drop(columns=['zip'],axis=1,inplace=True)

In [6]:
y_train_df=y_train_df.astype('float')

In [7]:
y_test_df['LABEL_bubbleStore_cnt_fst_agg'] = y_test_df['LABEL_bubbleStore_cnt_fst_agg'].astype('float')

In [59]:
X_train_df.head()

,college_cnt_fst_agg,walmart_cnt_fst_agg,Num_Mcdonalds_fst_agg,Num_Starbucks_fst_agg,Num_Teavana_fst_agg,population_total_fst_agg,pop_under19_fst_agg,pop_19To34_fst_agg,pop_35To54_fst_agg,pop_55AndOver_fst_agg,...,pop_householdIncome_under50K_fst_agg,pop_householdIncome_50Kto100K_fst_agg,pop_householdIncome_over100K_fst_agg,pop_belowPovertyThreshold_fst_agg,income_median_fst_agg,shopping_mall_cnt_fst_agg,pop_density_fst_agg,selfstorage_count_fst_agg,selfstorage_area_fst_agg,costco_count_fst_agg
0,0.001445,0.001094,0.001483,0.000000,0.0,0.004711,0.004084,0.004518,0.004117,0.004878,...,0.004722,0.004328,0.004533,0.004374,0.005502,0.000000,0.004532,0.000799,0.001659,0.003198
1,0.001827,0.000000,0.002181,0.000000,0.0,0.005549,0.005449,0.005383,0.005398,0.005684,...,0.005810,0.005619,0.005142,0.002455,0.005461,0.000000,0.004583,0.002293,0.001591,0.000000
2,0.003064,0.001548,0.002151,0.002223,0.0,0.004783,0.004467,0.005067,0.004748,0.004186,...,0.004834,0.005005,0.003586,-0.000547,0.005446,0.002211,0.004138,0.001131,0.000397,0.000000
3,0.003076,0.000445,0.001058,0.000639,0.0,0.005033,0.004775,0.004622,0.004973,0.004710,...,0.004683,0.004906,0.004777,0.003063,0.005975,0.001389,0.004949,0.000686,0.000744,0.000000
4,0.001202,0.000911,0.000844,0.001308,0.0,0.003475,0.003179,0.003063,0.003179,0.003363,...,0.003413,0.003544,0.001710,0.002341,0.005305,0.002051,0.005170,0.000000,0.000000,0.000000


## Building Model

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn.functional as F

In [68]:
class MyNN(nn.Module):
    def __init__(self, in_num,out_num):
        super(MyNN,self).__init__()
        self.flatten = nn.Flatten()
        self.in_node = in_num
        self.out_node = out_num
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_num,64),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16 ,out_num),
        )
        
    def forward(self,x):
        logits = self.linear_relu_stack(x)
        return logits

In [10]:
#the value is so small 
# scale to a proper value
y_train_100x_df = y_train_df*100

In [12]:
X_train_100x_df = X_train_df*100

In [13]:
X_test_100x_df = X_test_df*100

In [14]:
X_train_100x_df.head()

,college_cnt_fst_agg,walmart_cnt_fst_agg,Num_Mcdonalds_fst_agg,Num_Starbucks_fst_agg,Num_Teavana_fst_agg,population_total_fst_agg,pop_under19_fst_agg,pop_19To34_fst_agg,pop_35To54_fst_agg,pop_55AndOver_fst_agg,...,pop_householdIncome_under50K_fst_agg,pop_householdIncome_50Kto100K_fst_agg,pop_householdIncome_over100K_fst_agg,pop_belowPovertyThreshold_fst_agg,income_median_fst_agg,shopping_mall_cnt_fst_agg,pop_density_fst_agg,selfstorage_count_fst_agg,selfstorage_area_fst_agg,costco_count_fst_agg
0,0.144459,0.109441,0.148328,0.000000,0.0,0.471078,0.408411,0.451771,0.411742,0.487829,...,0.472217,0.432816,0.453264,0.437378,0.550199,0.000000,0.453200,0.079940,0.165941,0.319804
1,0.182720,0.000000,0.218137,0.000000,0.0,0.554857,0.544910,0.538296,0.539808,0.568352,...,0.580966,0.561905,0.514250,0.245542,0.546095,0.000000,0.458318,0.229312,0.159059,0.000000
2,0.306443,0.154773,0.215086,0.222262,0.0,0.478329,0.446723,0.506680,0.474807,0.418609,...,0.483410,0.500492,0.358580,-0.054743,0.544593,0.221056,0.413774,0.113052,0.039667,0.000000
3,0.307608,0.044525,0.105771,0.063940,0.0,0.503326,0.477477,0.462177,0.497255,0.471029,...,0.468263,0.490597,0.477745,0.306349,0.597513,0.138929,0.494906,0.068564,0.074379,0.000000
4,0.120197,0.091060,0.084364,0.130767,0.0,0.347522,0.317921,0.306335,0.317908,0.336258,...,0.341296,0.354397,0.170958,0.234081,0.530492,0.205147,0.516993,0.000000,0.000000,0.000000


In [23]:
# split into train and valid data
X_train, X_valid,y_train,y_valid = train_test_split(X_train_100x_df,y_train_100x_df,test_size=0.3)

In [26]:
X_train.head()

,college_cnt_fst_agg,walmart_cnt_fst_agg,Num_Mcdonalds_fst_agg,Num_Starbucks_fst_agg,Num_Teavana_fst_agg,population_total_fst_agg,pop_under19_fst_agg,pop_19To34_fst_agg,pop_35To54_fst_agg,pop_55AndOver_fst_agg,...,pop_householdIncome_under50K_fst_agg,pop_householdIncome_50Kto100K_fst_agg,pop_householdIncome_over100K_fst_agg,pop_belowPovertyThreshold_fst_agg,income_median_fst_agg,shopping_mall_cnt_fst_agg,pop_density_fst_agg,selfstorage_count_fst_agg,selfstorage_area_fst_agg,costco_count_fst_agg
14079,0.000000,0.138433,0.320631,0.099398,0.0,0.505865,0.503895,0.469516,0.490589,0.494531,...,0.444986,0.475336,0.538390,0.329633,0.565073,0.000000,0.463349,0.303351,0.388927,0.000000
10286,0.266213,0.247963,0.364715,0.238030,0.0,0.613156,0.591771,0.597830,0.607744,0.606114,...,0.587360,0.605625,0.578810,0.354317,0.585598,0.335872,0.627128,0.262632,0.306297,0.092365
5515,0.115825,0.000000,0.162590,0.882074,0.0,0.326547,0.286897,0.252164,0.300052,0.331190,...,0.320735,0.301788,0.278291,-0.088607,0.558615,0.000000,0.114991,0.000000,0.000000,0.000000
2821,0.144963,0.105147,0.189210,0.000000,0.0,0.453926,0.439373,0.418412,0.439166,0.439389,...,0.478114,0.420117,0.399975,0.294279,0.524219,0.000000,0.328839,0.076804,0.059415,0.000000
6318,0.000000,0.309546,0.203045,0.203597,0.0,0.387818,0.337206,0.336167,0.355707,0.405380,...,0.369075,0.376098,0.373862,0.766708,0.514980,0.091965,0.242794,0.000000,0.000000,0.406464


In [20]:
X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)
X_valid_tensor = torch.FloatTensor(X_valid.values)
y_valid_tensor = torch.FloatTensor(y_valid.values)

In [21]:
X_test_tensor = torch.FloatTensor(X_test_100x_df.values)

In [69]:
model = MyNN(32,1)
print(model)

MyNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=32, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


## Define Loss function and optimizer

In [70]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.005,weight_decay=0.01)
train_loss=[]
valid_loss=[]
min_valid_loss=99999

## Model Training

In [71]:
#train model
for epoch in range(5000):
    optimizer.zero_grad()
    train_outputs = model(X_train_tensor)
    train_loss = criterion(train_outputs,y_train_tensor)
    train_loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        valid_outputs = model(X_valid_tensor)
        valid_loss = criterion(valid_outputs,y_valid_tensor)
    
    if min_valid_loss > valid_loss:
        min_valid_loss = valid_loss
        torch.save(
            {
                'epoch':epoch,
                'model_state_dict':model.state_dict(),
                'optimizer_state_dict':optimizer.state_dict(),
                'train_loss':train_loss,
                'valid_loss':valid_loss
            },"model.pt"
        )
    if epoch%100==0:
        print("train loss: {} valid loss: {}".format(train_loss,valid_loss))

train loss: 0.18915106356143951 valid loss: 0.14145009219646454
train loss: 0.08405731618404388 valid loss: 0.10569027066230774
train loss: 0.08395204693078995 valid loss: 0.10561441630125046
train loss: 0.08394769579172134 valid loss: 0.10561791807413101
train loss: 0.08395790308713913 valid loss: 0.10557813942432404
train loss: 0.0839373767375946 valid loss: 0.10568011552095413
train loss: 0.08395665138959885 valid loss: 0.10554580390453339
train loss: 0.08390717208385468 valid loss: 0.1055593341588974
train loss: 0.08390570431947708 valid loss: 0.10561024397611618
train loss: 0.08389412611722946 valid loss: 0.10560280084609985
train loss: 0.08390030264854431 valid loss: 0.10551843047142029
train loss: 0.08389072120189667 valid loss: 0.10554042458534241
train loss: 0.0839071273803711 valid loss: 0.10548919439315796
train loss: 0.08390017598867416 valid loss: 0.10552078485488892
train loss: 0.08389648050069809 valid loss: 0.10549795627593994
train loss: 0.08389604091644287 valid loss:

## Prediction

In [74]:
test_prediction_tensor = model(X_test_tensor)

In [39]:
test_target = y_test_df['LABEL_bubbleStore_cnt_fst_agg']*100

In [35]:
values=criterion(test_prediction,torch.FloatTensor(test_target.values))

F:\Users\zzguk\Anaconda3\envs\NTUHW\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([7366])) that is different to the input size (torch.Size([7366, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [36]:
np.sqrt(values.item()/10000)

0.003762879110481246

In [44]:
test_target_tensor = torch.FloatTensor(test_target.values)

In [76]:
test_prediction = test_prediction_tensor.detach().numpy()

## Error Calculation

In [79]:
np.square(test_target-test_prediction).mean()/10000

1.4168115533259323e-05

In [87]:
np.sqrt(np.square(test_target-test_prediction).mean())/100

0.0037640557292977646

In [85]:
#R2 calculation
var_mean = np.square(test_target-test_target.mean()).mean()

In [86]:
var_mean

0.11850210955148112

In [82]:
var_pred = np.square(test_target-test_prediction).mean()

In [83]:
var_pred

0.14168115533259323

In [90]:
1-(var_pred/var_mean)

-0.1956002797658416

In [92]:
np.fabs(test_target-test_prediction).mean()/100

0.0018175983606310909